<a href="https://colab.research.google.com/github/absolutelydawn/TestNeoX/blob/main/Learn2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# 관련 라이브러리 설치
!pip install transformers pandas

# GitHub 저장소 클론
!git clone https://github.com/songys/Chatbot_data.git

import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, GPTNeoForCausalLM, AdamW
from transformers import get_linear_schedule_with_warmup

# 데이터 전처리 및 로드
class CustomDataset(Dataset):
    def __init__(self, data, tokenizer):
        # 데이터 전처리 및 토큰화
        self.tokenized_data = tokenizer(data, truncation=True, padding=True, return_tensors="pt")

    def __len__(self):
        return len(self.tokenized_data)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.tokenized_data.items()}
        return item

# CSV 파일 읽기 및 데이터 로드
chatbot_data = pd.read_csv("Chatbot_data/ChatbotData.csv")
data = chatbot_data['Q'].tolist()
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B")

# 패딩 토큰 설정
tokenizer.pad_token = tokenizer.eos_token

dataset = CustomDataset(data, tokenizer)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

# 모델 및 학습 설정
model = GPTNeoForCausalLM.from_pretrained("EleutherAI/gpt-neo-1.3B")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = AdamW(model.parameters(), lr=1e-5)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=1, num_training_steps=-1)

num_epochs = 5

# 전이학습 진행
for epoch in range(num_epochs):
    model.train()
    for idx, batch in enumerate(dataloader):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

        if idx % 10 == 0:
            print(f"Epoch: {epoch}, Step: {idx}, Loss: {loss.item()}")


fatal: destination path 'Chatbot_data' already exists and is not an empty directory.


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


AttributeError: ignored

In [3]:
# 위 코드 수정 : Dataset 클래스에서 self.tokenized_data 대신 self.tokenized_data['input_ids']만 사용
# get_linear_schedule_with_warmup 함수에서 num_training_steps를 지정

# 용량 작은 문제 해결 안됨

# 관련 라이브러리 설치
!pip install transformers pandas

# GitHub 저장소 클론
!git clone https://github.com/songys/Chatbot_data.git

import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, GPTNeoForCausalLM, AdamW
from transformers import get_linear_schedule_with_warmup

# 데이터 전처리 및 로드
class CustomDataset(Dataset):
    def __init__(self, data, tokenizer):
        # 데이터 전처리 및 토큰화
        self.tokenized_data = tokenizer(data, truncation=True, padding=True, return_tensors="pt")["input_ids"]

    def __len__(self):
        return len(self.tokenized_data)

    def __getitem__(self, idx):
        item = self.tokenized_data[idx]
        return item

# CSV 파일 읽기 및 데이터 로드
chatbot_data = pd.read_csv("Chatbot_data/ChatbotData.csv")
data = chatbot_data['Q'].tolist()
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B")

# 패딩 토큰 설정
tokenizer.pad_token = tokenizer.eos_token

dataset = CustomDataset(data, tokenizer)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

# 모델 및 학습 설정
model = GPTNeoForCausalLM.from_pretrained("EleutherAI/gpt-neo-1.3B")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = AdamW(model.parameters(), lr=1e-5)
epochs = 5
total_steps = len(dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=100, num_training_steps=total_steps)

num_epochs = 5

# 전이학습 진행
for epoch in range(num_epochs):
    model.train()
    for idx, batch in enumerate(dataloader):
        inputs = batch.to(device)
        outputs = model(input_ids=inputs, labels=inputs)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

        if idx % 10 == 0:
            print(f"Epoch: {epoch}, Step: {idx}, Loss: {loss.item()}")


fatal: destination path 'Chatbot_data' already exists and is not an empty directory.


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


OutOfMemoryError: ignored

In [ ]:
# 위 코드에서 배치사이즈 1, GPT-Neo 350M 모델로 변경
# 데이터셋 크기 줄임 : data = chatbot_data['Q'].tolist()[:1000]

# 관련 라이브러리 설치
!pip install transformers pandas

# GitHub 저장소 클론
!git clone https://github.com/songys/Chatbot_data.git

import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, GPTNeoForCausalLM, AdamW
from transformers import get_linear_schedule_with_warmup

# 데이터 전처리 및 로드
class CustomDataset(Dataset):
    def __init__(self, data, tokenizer):
        # 데이터 전처리 및 토큰화
        self.tokenized_data = tokenizer(data, truncation=True, padding=True, return_tensors="pt")["input_ids"]

    def __len__(self):
        return len(self.tokenized_data)

    def __getitem__(self, idx):
        item = self.tokenized_data[idx]
        return item

# CSV 파일 읽기 및 데이터 로드
chatbot_data = pd.read_csv("Chatbot_data/ChatbotData.csv")
data = chatbot_data['Q'].tolist()[:1000]
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-2.7B")

# 패딩 토큰 설정
tokenizer.pad_token = tokenizer.eos_token

dataset = CustomDataset(data, tokenizer)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

# 모델 및 학습 설정
model = GPTNeoForCausalLM.from_pretrained("EleutherAI/gpt-neo-2.7B")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = AdamW(model.parameters(), lr=1e-5)
epochs = 5
total_steps = len(dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=100, num_training_steps=total_steps)

num_epochs = 5

# 전이학습 진행
for epoch in range(num_epochs):
    model.train()
    for idx, batch in enumerate(dataloader):
        inputs = batch.to(device)
        outputs = model(input_ids=inputs, labels=inputs)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

        if idx % 10 == 0:
            print(f"Epoch: {epoch}, Step: {idx}, Loss: {loss.item()}")



fatal: destination path 'Chatbot_data' already exists and is not an empty directory.
